In [12]:
!pip3 install numpy --upgrade
!pip3 install scipy --upgrade

In [13]:
import psutil
import logging
from importlib import reload

import fft_project
from fft_project import base

import numpy as np
from scipy.fft import fft2, ifft2

In [14]:
'''Below are the testcases, where we controlled the multithreading and creating copies
of our version of 2DFFT versus Scipy fft2'''

#Printing out Current CPU Info
print("="*40, "CPU Info", "="*40)
print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", psutil.cpu_count(logical=True))
cpufreq = psutil.cpu_freq()
print(f"Max Frequency: {cpufreq.max:.2f}Mhz")
print(f"Min Frequency: {cpufreq.min:.2f}Mhz")
print(f"Current Frequency: {cpufreq.current:.2f}Mhz")

======================================== CPU Info ========================================
Physical cores: 4
Total cores: 8
Max Frequency: 2800.00Mhz
Min Frequency: 2800.00Mhz
Current Frequency: 2800.00Mhz


In [15]:
def random_matrix(n, m):
    logging.info(f'Matrix shape: {n}x{m}')
    result = np.zeros((n, m), dtype=np.complex128)
    result.real = np.random.rand(n, m)
    return result

reload(logging)
base.prepare_logger()


Multithreading enabled and type conversions

In [22]:
def fft_cpp_impl_test():
    result = random_matrix(4096, 4096 * 2)
    print('Scipy Time:')
    %timeit -o scipy.fft.fft2(result, workers= -1)
    print('Our Time:')
    %timeit -o fft_project.fft2d(result, return_copy=True, use_threads=True)
    expected = fft2(result, workers= -1)
    result = fft_project.fft2d(result, return_copy=True, use_threads=True)
    correctness = np.allclose(result, expected, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

def inverse_test():
    a = random_matrix(4096 * 2, 4096)
    transformed = fft_project.fft2d(a, return_copy=True, use_threads=True)
    print('Scipy Time:')
    %timeit -o scipy.fft.ifft2(transformed, workers= -1)
    print('Our Time:')
    %timeit -o fft_project.fft2d(transformed, return_copy=True, use_threads=True, inverse=True)
    inversed_scipy = ifft2(transformed, workers= -1)
    inversed_our = fft_project.fft2d(
    transformed, return_copy=True, use_threads=True, inverse=True
    )
    correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

fft_cpp_impl_test()
inverse_test()


[INFO]  Matrix shape: 4096x8192


Scipy Time:
483 ms ± 76.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Our Time:
1.02 s ± 32.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Is the result correct: True
[INFO]  Matrix shape: 8192x4096


Scipy Time:
492 ms ± 21.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Our Time:
1.05 s ± 11.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Is the result correct: True


With multithreading enabled, no type conversions

In [21]:
def fft_cpp_impl_test():
    result = random_matrix(4096, 4096 * 2)
    print('Scipy Time:')
    %timeit -o scipy.fft.fft2(result, workers =-1)
    print('Our Time:')
    %timeit -o fft_project.fft2d(result, return_copy=False, use_threads=True)
    expected = fft2(result, workers= -1)
    result = fft_project.fft2d(result, return_copy=False, use_threads=True)
    correctness = np.allclose(result, expected, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

def inverse_test():
    a = random_matrix(4096 * 2, 4096)
    transformed = fft_project.fft2d(a, return_copy=False, use_threads=True)
    print('Scipy Time:')
    %timeit -o scipy.fft.ifft2(transformed, workers= -1)
    print('Our Time:')
    %timeit -o fft_project.fft2d(transformed, return_copy=False, use_threads=True, inverse=True)
    inversed_scipy = ifft2(transformed, workers= -1)
    inversed_our = fft_project.fft2d(
    transformed, return_copy=False, use_threads=True, inverse=True
    )
    correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

fft_cpp_impl_test()
inverse_test()


[INFO]  Matrix shape: 4096x8192


Scipy Time:
648 ms ± 74.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Our Time:
698 ms ± 37.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Is the result correct: True
[INFO]  Matrix shape: 8192x4096


Scipy Time:
589 ms ± 41.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Our Time:
689 ms ± 49.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Is the result correct: True


No multithreading, with type conversions

In [20]:
def fft_cpp_impl_test():
    result = random_matrix(4096, 4096 * 2)
    print('Scipy Time:')
    %timeit -o scipy.fft.fft2(result)
    print('Our Time:')
    %timeit -o fft_project.fft2d(result, return_copy=True, use_threads=False)
    expected = fft2(result)
    result = fft_project.fft2d(result, return_copy=True, use_threads=False)
    correctness = np.allclose(result, expected, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

def inverse_test():
    a = random_matrix(4096 * 2, 4096)
    transformed = fft_project.fft2d(a, return_copy=True, use_threads=False)
    print('Scipy Time:')
    %timeit -o scipy.fft.ifft2(transformed)
    print('Our Time:')
    %timeit -o fft_project.fft2d(transformed, return_copy=True, use_threads=False, inverse=True)
    inversed_scipy = ifft2(transformed)
    inversed_our = fft_project.fft2d(
    transformed, return_copy=True, use_threads=False, inverse=True
    )
    correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

fft_cpp_impl_test()
inverse_test()

[INFO]  Matrix shape: 4096x8192


Scipy Time:
1.79 s ± 86.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Our Time:
1.53 s ± 13.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Is the result correct: True
[INFO]  Matrix shape: 8192x4096


Scipy Time:
1.64 s ± 20.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Our Time:
1.73 s ± 150 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Is the result correct: True


No multithreading, no type conversions

In [19]:
def fft_cpp_impl_test():
    result = random_matrix(4096, 4096 * 2)
    print('Scipy Time:')
    %timeit -o scipy.fft.fft2(result)
    print('Our Time:')
    %timeit -o fft_project.fft2d(result, return_copy=False, use_threads=False)
    expected = fft2(result)
    result = fft_project.fft2d(result, return_copy=False, use_threads=False)
    correctness = np.allclose(result, expected, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

def inverse_test():
    a = random_matrix(4096 * 2, 4096)
    transformed = fft_project.fft2d(a, return_copy=False, use_threads=False)
    print('Scipy Time:')
    %timeit -o scipy.fft.ifft2(transformed)
    print('Our Time:')
    %timeit -o fft_project.fft2d(transformed, return_copy=False, use_threads=False, inverse=True)
    inversed_scipy = ifft2(transformed)
    inversed_our = fft_project.fft2d(
    transformed, return_copy=False, use_threads=False, inverse=True)
    correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

fft_cpp_impl_test()
inverse_test()

[INFO]  Matrix shape: 4096x8192


Scipy Time:
1.8 s ± 92.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Our Time:
1.21 s ± 69.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Is the result correct: True
[INFO]  Matrix shape: 8192x4096


Scipy Time:
1.74 s ± 101 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Our Time:
1.2 s ± 73.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Is the result correct: True
